# 使用Sequential API构建回归MLP

本教程演示如何使用Keras的Sequential API构建多层感知机(MLP)来解决回归问题。

## 学习目标

1. 掌握Sequential API的基本用法
2. 理解回归任务的模型构建流程
3. 学会数据预处理和特征标准化
4. 掌握模型训练和评估方法

## 数据集介绍

使用California Housing数据集，该数据集包含1990年加州各区块的房价信息：
- 8个特征：收入中位数、房龄、房间数等
- 目标变量：房价中位数（单位：10万美元）

## 1. 环境配置

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 设置随机种子确保结果可复现
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print(f"TensorFlow版本: {tf.__version__}")
print(f"Keras版本: {keras.__version__}")

## 2. 数据加载与探索

In [ ]:
# 加载California Housing数据集
housing = fetch_california_housing()

# 查看数据集基本信息
print("数据集描述:")
print("="*50)
print(f"特征数量: {housing.data.shape[1]}")
print(f"样本数量: {housing.data.shape[0]}")
print(f"\n特征名称: {housing.feature_names}")
print(f"\n目标变量: 房价中位数 (单位: 10万美元)")

In [ ]:
# 创建DataFrame便于数据探索
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['MedHouseVal'] = housing.target

print("数据统计信息:")
df.describe()

## 3. 数据预处理

### 数据集划分策略

将数据划分为三部分：
- **训练集 (60%)**: 用于模型参数学习
- **验证集 (20%)**: 用于超参数调优和早停
- **测试集 (20%)**: 用于最终模型评估

In [ ]:
# 第一次划分：分离出测试集
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, 
    test_size=0.2, 
    random_state=RANDOM_SEED
)

# 第二次划分：从剩余数据中分离出验证集
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, 
    test_size=0.25,  # 0.25 * 0.8 = 0.2
    random_state=RANDOM_SEED
)

print(f"训练集大小: {X_train.shape[0]}")
print(f"验证集大小: {X_valid.shape[0]}")
print(f"测试集大小: {X_test.shape[0]}")

### 特征标准化

神经网络对输入特征的尺度敏感，需要进行标准化处理：
$$x_{\text{scaled}} = \frac{x - \mu}{\sigma}$$

**注意**: 标准化参数(均值和标准差)只能从训练集计算，然后应用到验证集和测试集。

In [ ]:
# 创建标准化器
scaler = StandardScaler()

# 在训练集上拟合并转换
X_train = scaler.fit_transform(X_train)

# 在验证集和测试集上只进行转换（使用训练集的参数）
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

print("标准化后的训练数据统计:")
print(f"均值: {X_train.mean(axis=0).round(2)}")
print(f"标准差: {X_train.std(axis=0).round(2)}")

## 4. 构建MLP模型

### Sequential API介绍

Sequential API是Keras中最简单的模型构建方式，适用于层的线性堆叠。

### 模型架构

```
输入层 (8个特征)
    ↓
隐藏层1 (30个神经元, ReLU激活)
    ↓
输出层 (1个神经元, 线性激活)
```

In [ ]:
# 方式1：使用add()方法逐层添加
model = keras.Sequential()

# 添加输入层和第一个隐藏层
# input_shape指定输入特征的维度，不包括batch_size
model.add(keras.layers.Dense(
    units=30,                      # 神经元数量
    activation='relu',             # ReLU激活函数
    input_shape=X_train.shape[1:]  # 输入形状: (8,)
))

# 添加输出层
# 回归任务输出层使用线性激活（默认）
model.add(keras.layers.Dense(units=1))

# 查看模型结构
model.summary()

In [ ]:
# 方式2：在构造函数中直接传入层列表（等效写法）
# model = keras.Sequential([
#     keras.layers.Dense(30, activation='relu', input_shape=X_train.shape[1:]),
#     keras.layers.Dense(1)
# ])

## 5. 编译模型

编译时需要指定：
- **损失函数**: 回归任务使用均方误差(MSE)
- **优化器**: SGD、Adam等
- **评估指标**: MAE更具可解释性

In [ ]:
model.compile(
    loss='mean_squared_error',      # MSE损失函数
    optimizer='sgd',                # 随机梯度下降优化器
    metrics=['mean_absolute_error'] # 额外监控MAE指标
)

## 6. 训练模型

In [ ]:
# 训练模型
history = model.fit(
    X_train, y_train,
    epochs=50,                              # 训练轮数
    batch_size=32,                          # 批量大小
    validation_data=(X_valid, y_valid),     # 验证数据
    verbose=1                               # 显示训练进度
)

## 7. 可视化训练过程

In [ ]:
def plot_learning_curves(history, metrics=['loss']):
    """
    绘制学习曲线
    
    Parameters:
    -----------
    history : keras.callbacks.History
        训练历史对象
    metrics : list
        要绘制的指标列表
    """
    n_metrics = len(metrics)
    fig, axes = plt.subplots(1, n_metrics, figsize=(6*n_metrics, 4))
    
    if n_metrics == 1:
        axes = [axes]
    
    for ax, metric in zip(axes, metrics):
        ax.plot(history.history[metric], label=f'Training {metric}')
        ax.plot(history.history[f'val_{metric}'], label=f'Validation {metric}')
        ax.set_xlabel('Epoch')
        ax.set_ylabel(metric.upper())
        ax.set_title(f'Learning Curve - {metric.upper()}')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# 绘制损失曲线和MAE曲线
plot_learning_curves(history, metrics=['loss', 'mean_absolute_error'])

In [ ]:
# 使用pandas绘制（简洁方式）
pd.DataFrame(history.history).plot(figsize=(10, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1.5)
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.title('Training History')
plt.show()

## 8. 模型评估

In [ ]:
# 在测试集上评估模型
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)

print("测试集评估结果:")
print(f"MSE (均方误差): {test_loss:.4f}")
print(f"RMSE (均方根误差): {np.sqrt(test_loss):.4f}")
print(f"MAE (平均绝对误差): {test_mae:.4f}")
print(f"\n解释: 模型预测房价的平均误差约为 ${test_mae*100000:.0f}")

## 9. 使用模型进行预测

In [ ]:
# 对新样本进行预测
X_new = X_test[:5]  # 取测试集前5个样本
y_pred = model.predict(X_new, verbose=0)

print("预测结果对比:")
print("="*40)
for i in range(len(X_new)):
    actual = y_test[i]
    predicted = y_pred[i][0]
    error = abs(actual - predicted)
    print(f"样本{i+1}: 实际={actual:.3f}, 预测={predicted:.3f}, 误差={error:.3f}")

In [ ]:
# 可视化预测结果
y_pred_all = model.predict(X_test, verbose=0).flatten()

plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred_all, alpha=0.5, s=10)
plt.plot([0, 5], [0, 5], 'r--', label='Perfect Prediction')
plt.xlabel('Actual House Value (100k $)')
plt.ylabel('Predicted House Value (100k $)')
plt.title('Actual vs Predicted House Values')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.xlim(0, 5)
plt.ylim(0, 5)
plt.tight_layout()
plt.show()

## 小结

### 回归任务的关键点

1. **数据预处理**: 特征标准化对神经网络至关重要
2. **输出层设计**: 回归任务使用单个神经元，无激活函数（线性输出）
3. **损失函数**: 通常使用MSE或MAE
4. **评估指标**: RMSE和MAE更具可解释性

### Sequential API适用场景

- 层的简单线性堆叠
- 单输入单输出模型
- 快速原型开发

### 后续改进方向

1. 增加隐藏层数量和神经元
2. 添加正则化（Dropout、L2）
3. 使用更优的优化器（Adam）
4. 应用早停和学习率调度